In [1]:
import pandas as pd
from classes.WindAPI import *
from sqlalchemy import *
from sqlalchemy.orm import Session
from create_ref_mariadb import mariadb_connection, windturbines
from create_ref_mongodb import mongodb_connection
import pandas as pd
import plotly.express as px
from datetime import datetime, timedelta
import pytz

In [2]:
def fetch_windturbines_data(engine):
    
    df = pd.read_sql(
    """
    SELECT wf.windfarm_id, wf.code AS wfCode, wt.windturbine_id, wt.code AS wtCode, wt.last_scada_update
    FROM mariadb_itw.windturbines AS wt
    INNER JOIN mariadb_itw.windfarms AS wf
        ON wf.windfarm_id = wt.windfarm_id
    WHERE wt.last_scada_update IS NOT NULL
    ORDER BY wf.code ASC, wt.code ASC
    ;
    """,
        con=engine
    )
    

    return df

client_mongodb = mongodb_connection()
engine_mariadb = mariadb_connection()
turbinesDF = fetch_windturbines_data(engine_mariadb)
turbinesDF.head()

tz = pytz.timezone('Europe/Paris')
current_datetime = datetime.now(tz)
lastday_datetime = current_datetime - timedelta(hours=24)

scada_col = client_mongodb.scada
scada_data = scada_col.find({
    "log_date": {'$gte': lastday_datetime}
}).sort("log_date", 1)

df = pd.DataFrame(list(scada_data))

for turbine in turbinesDF:
    if (len(df[df['wind_turbine'] == turbine.windturbine_id]) > 0):
        fig = px.line(df[df['wind_turbine'] == turbine.windturbine_id], x="log_date", y="active_power", title=turbine.wtCode)
        fig.show()
    

: 